In [1]:
import os
import sys
import json
import logging
from dotenv import load_dotenv 
load_dotenv()
sys.path.append(os.getenv("PROJECT_PATH"))
from src.utils.azure_client import AzureClient, get_answer

from src.Evaluators.gpt_evaluator import GPTEvaluator

from src.Obfuscators.few_prompt_obfuscator import FewPromptsObfuscator
from src.Obfuscators.random_text import RandomText
from src.Obfuscators.totally_random_emoji import TotallyRandomEmoji


'The capital of France is Paris.'

In [51]:
disable_httpx_log = logging.getLogger("httpx")
disable_httpx_log.setLevel(logging.WARNING)
logging.basicConfig(filename="random_log", level=logging.INFO)
logger = logging.getLogger(__name__)

In [52]:
two_prompt_1_path = os.path.join(os.getenv("PROJECT_PATH"),"src","Obfuscators","prompts","two_querries", "first_querry.txt")
two_prompt_2_path = os.path.join(os.getenv("PROJECT_PATH"),"src","Obfuscators","prompts","two_querries", "second_querry_random_emoji.txt")

azure_llm_wrapper_factory = lambda :AzureClient("azure_client",  "../", "gpt-4o-2024-05-13", 0.0)
with open(two_prompt_1_path, 'r', encoding='utf-8') as file:
    two_prompt_1 = file.read()
with open(two_prompt_2_path, 'r', encoding='utf-8') as file:
    two_prompt_2 = file.read()


cpprefix = "Do not explain the emojis in your answer.\n"
smart_random_emoji = lambda : FewPromptsObfuscator([two_prompt_1,two_prompt_2], azure_llm_wrapper_factory, logger, cpprefix)
totally_random_emoji = lambda : TotallyRandomEmoji()
random_text = lambda : RandomText(azure_llm_wrapper_factory)

gpt_evaluator = GPTEvaluator(logger, "..")

GPT EVALUATOR INIT


In [53]:
data_to_use = "RESULTS_2024-08-07_10_46_47.771852gpt_metric_new_test.json"
inputfile_path = os.path.join(os.getenv("PROJECT_PATH"),"data",data_to_use)
with open(inputfile_path, 'r') as file:
    data = json.load(file)

In [54]:
original_prompt = "How does the use of machine learning algorithms in predictive analytics enhance decision-making processes in businesses, and what are some common challenges faced during implementation? BE CONCISE"
original_answer = "Machine learning algorithms enhance decision-making in businesses by analyzing large datasets to identify patterns, predict outcomes, and provide actionable insights. This leads to more informed, data-driven decisions, improved efficiency, and competitive advantages.\n\nCommon challenges during implementation include:\n\n1. **Data Quality**: Incomplete or inaccurate data can lead to poor model performance.\n2. **Complexity**: Developing and tuning models requires specialized skills and knowledge.\n3. **Integration**: Incorporating machine learning systems into existing workflows and IT infrastructure can be difficult.\n4. **Scalability**: Ensuring models perform well as data volume grows.\n5. **Bias**: Models can perpetuate or amplify existing biases in data.\n6. **Cost**: High initial investment in technology and talent.\n\nAddressing these challenges requires careful planning, ongoing monitoring, and a commitment to data governance and ethical practices."

In [58]:
obfuscated_prompt = totally_random_emoji().obfuscate(original_prompt)
print(obfuscated_prompt)

How does the use of machine learning algorithms in predictive analytics enhance decision-making processes in businesses, and what are some common challenges faced during implementation? BE CONCISE


In [56]:
print(get_answer("What is the capital of France?",model = "gpt-35-turbo"))

AuthenticationError: Error code: 401 - {'statusCode': 401, 'message': 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.'}

In [ ]:
results_arr = []
for i in range (1):
    obfuscated_prompt = totally_random_emoji().obfuscate(original_prompt)
    prompt_evaluation = gpt_evaluator.evaluate_prompt(original_prompt, obfuscated_prompt)
    answer = AzureClient.get_answer(original_prompt)
    answer_evaluation = gpt_evaluator.evaluate_prompt(original_answer, answer)
    print("""
obfuscated_prompt: {obfuscated_prompt}
prompt_evaluation: {prompt_evaluation}
answer: {answer}
answer_evaluation: {answer_evaluation}
          """)
    results_arr.append(prompt_evaluation)

AuthenticationError: Error code: 401 - {'statusCode': 401, 'message': 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.'}

In [5]:
import difflib

def get_different_words(text1, text2):
    # Tokenize the texts into words
    words1 = text1.split()
    words2 = text2.split()

    # Use difflib to find differences
    diff = difflib.ndiff(words1, words2)

    # Extract the words that are different
    different_words = [word[2:] for word in diff if word.startswith('- ') or word.startswith('+ ')]

    return different_words

# Example usage
text1= "How does the use of machine learning algorithms in predictive analytics enhance decision-making processes in businesses, and what are some common challenges faced during implementation?"
text2 = "How does the use of 🍀🚀🎨 in 🎉✨🌍 enhance 🌟🍀 in 🌍📚, and what are some common challenges faced during implementation?" 

differences = get_different_words(text1, text2)
print(differences)


['simple', 'sample', 'two', 'three']
